In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('bug_dataset_v1.0_20250327_160057.csv')

# Data preprocessing
# For bug detection model
df['is_buggy'] = df['label'].apply(lambda x: 1 if x == 1 else 0)

# For bug fixing model, we'll use only the buggy examples
buggy_df = df[df['is_buggy'] == 1].copy()
buggy_df['bug_type'] = buggy_df['bug_type'].fillna('unknown')

# Feature extraction for bug detection
vectorizer = TfidfVectorizer(max_features=1000, stop_words=None, token_pattern=r'\b\w+\b|\S+')
X = vectorizer.fit_transform(df['code'])
y = df['is_buggy']

# Split data for bug detection
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train bug detection model
bug_detection_model = RandomForestClassifier(n_estimators=100, random_state=42)
bug_detection_model.fit(X_train, y_train)

# Evaluate bug detection
print("Bug Detection Model Performance:")
print(classification_report(y_test, bug_detection_model.predict(X_test)))

# Now let's create a bug fixing model
# We'll create a mapping from buggy code to fixed code based on the original_hash
fixed_code_map = {}
for _, row in df[df['label'] == 0].iterrows():
    func_name = row['code'].split('(')[0].split()[-1]  # Extract function name
    fixed_code_map[func_name] = row['code']

# For bug fixing, we'll create a model to predict the bug type first
bug_type_vectorizer = TfidfVectorizer(max_features=500, stop_words=None, token_pattern=r'\b\w+\b|\S+')
X_bug = bug_type_vectorizer.fit_transform(buggy_df['code'])
y_bug_type = buggy_df['bug_type']

# Split data for bug type classification
X_bt_train, X_bt_test, y_bt_train, y_bt_test = train_test_split(
    X_bug, y_bug_type, test_size=0.2, random_state=42
)

# Train bug type classifier
bug_type_model = RandomForestClassifier(n_estimators=100, random_state=42)
bug_type_model.fit(X_bt_train, y_bt_train)

# Evaluate bug type classification
print("\nBug Type Classification Performance:")
print(classification_report(y_bt_test, bug_type_model.predict(X_bt_test)))

# Bug fixing functions
def detect_bug(code):
    """Detect if code contains a bug"""
    vec = vectorizer.transform([code])
    return bug_detection_model.predict(vec)[0] == 1

def classify_bug_type(code):
    """Classify the type of bug in the code"""
    vec = bug_type_vectorizer.transform([code])
    return bug_type_model.predict(vec)[0]

def fix_missing_colon(code):
    """Fix missing colon in function definition"""
    lines = code.split('\n')
    def_line = lines[0]
    if ')' in def_line and ':' not in def_line:
        def_line = def_line + ':'
    return '\n'.join([def_line] + lines[1:])

def fix_missing_paren(code):
    """Fix missing parentheses in function definition"""
    lines = code.split('\n')
    def_line = lines[0]
    if 'def ' in def_line and '(' not in def_line:
        # Extract function name and parameters
        parts = def_line.split()
        func_name = parts[1].split('(')[0]
        params = parts[1].replace(func_name, '').strip()
        if params.endswith(':'):
            params = params[:-1]
        # Reconstruct the definition
        def_line = f"def {func_name}({params}):"
    return '\n'.join([def_line] + lines[1:])

def fix_name_error(code):
    """Fix name errors (like 'retrun' instead of 'return')"""
    return code.replace('retrun', 'return')

def fix_type_error(code):
    """Attempt to fix type errors (this is more complex)"""
    # This is a simple heuristic - in a real scenario you'd need more sophisticated analysis
    if "'string' b" in code:
        return code.replace("'string' b", "b")
    return code

def fix_buggy_code(code):
    """Main function to fix buggy code"""
    if not detect_bug(code):
        return code  # No bug detected
    
    bug_type = classify_bug_type(code)
    
    if bug_type == 'missing_colon':
        return fix_missing_colon(code)
    elif bug_type == 'missing_paren':
        return fix_missing_paren(code)
    elif bug_type == 'name_error':
        return fix_name_error(code)
    elif bug_type == 'type_error':
        return fix_type_error(code)
    else:
        # Try to find the correct version from the fixed_code_map
        func_name = code.split('(')[0].split()[-1]  # Extract function name
        return fixed_code_map.get(func_name, code)  # Return original if no fix found

#Test the bug fixer
test_cases = [
    # Missing colon
    """def variant_0_add(a, b)
    return a + b""",
    
    # Missing paren
    """def variant_0_adda, b:
    return a + b""",
    
    # Name error
    """def variant_0_add(a, b):
    retrun a + b""",
    
    # Type error
    """def variant_0_add(a, b):
    return a + 'string' b"""
]# 1. Missing colon
"def add(a, b)\n    return a + b"

# 2. Missing parentheses
"def adda, b:\n    return a + b"

# 3. Name error (retrun)
"def sqrt(x):\n    retrun x ** 0.5"

# 4. Type error (string + int)
"def add(a, b):\n    return a + '5' + b"

# 5. Incomplete exception handling
"def safe_divide(a, b):\n    try:\n        return a / b\n    except:\n        return None"


print("\nTesting Bug Fixer:")
for i, code in enumerate(test_cases):
    print(f"\nOriginal Code {i+1}:")
    print(code)
    print("\nFixed Code:")
    print(fix_buggy_code(code))
    print("\n" + "="*50)

Bug Detection Model Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        30
           1       1.00      1.00      1.00        70

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100


Bug Type Classification Performance:
               precision    recall  f1-score   support

missing_colon       1.00      1.00      1.00        19
missing_paren       1.00      1.00      1.00        24
   name_error       1.00      1.00      1.00        27
   type_error       1.00      1.00      1.00         6

     accuracy                           1.00        76
    macro avg       1.00      1.00      1.00        76
 weighted avg       1.00      1.00      1.00        76


Testing Bug Fixer:

Original Code 1:
def variant_0_add(a, b)
    return a + b

Fixed Code:
def variant_0_add(a, b):
    return a + b


Original Code 2:
def va